In [1]:
# Load packages
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('/data/home/stateun/data/fraud_data.csv')

In [3]:
data['FraudFound_P'].value_counts()

0    14497
1      923
Name: FraudFound_P, dtype: int64

In [4]:
923 / (14497+923)

0.05985732814526589

In [5]:
test1 = pd.read_csv('/data/home/stateun/results/test1_results.csv')
test2 = pd.read_csv('/data/home/stateun/results/test2_results.csv')
test3 = pd.read_csv('/data/home/stateun/results/test3_results.csv')
test4 = pd.read_csv('/data/home/stateun/results/test4_results.csv')
test5 = pd.read_csv('/data/home/stateun/results/test5_results.csv')

In [6]:
test = pd.concat([test1, test2, test3, test4, test5], axis = 0, ignore_index = True)
test = test.drop(['dataset'], axis = 1)

In [7]:
test

,model,AUC,ACC,Precision,Recall,F1_score,best_params
0,IGNnet,79.55,89.62,56.30,56.54,56.41,"alpha:0.01, sampling:smote, batch_size:64, epo..."
1,LogisticRegression,78.84,93.51,59.44,50.83,50.25,"{'C': 1.0, 'class_weight': None, 'dual': False..."
2,DecisionTree,52.59,86.77,51.90,52.59,52.08,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit..."
3,RandomForest,80.37,93.06,55.77,51.08,50.85,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
4,MLP,80.51,91.89,57.93,54.35,55.35,"{'activation': 'relu', 'alpha': 0.0001, 'batch..."
...,...,...,...,...,...,...,...
3672,DecisionTree,52.59,86.77,51.90,52.59,52.08,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit..."
3673,RandomForest,80.37,93.06,55.77,51.08,50.85,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
3674,MLP,80.51,91.89,57.93,54.35,55.35,"{'activation': 'relu', 'alpha': 0.0001, 'batch..."
3675,SVM,79.98,93.71,46.89,49.97,48.38,"{'C': 1.0, 'break_ties': False, 'cache_size': ..."


In [8]:
model_list = test['model'].unique()

In [9]:
IGNNet = test[test['model'] == 'IGNnet']
Table = IGNNet[(IGNNet['ACC'] >= 88) & (IGNNet['F1_score'] >= 60)].sort_values(by=['AUC', 'F1_score'], ascending=[False, False])
Table

,model,AUC,ACC,Precision,Recall,F1_score,best_params
3089,IGNnet,83.43,89.19,59.53,61.34,60.32,"alpha:0.03, sampling:borderline-smote, batch_s..."
2802,IGNnet,82.54,88.65,58.90,62.83,60.31,"alpha:0.01, sampling:borderline-smote, batch_s..."
2312,IGNnet,82.05,89.23,59.76,61.67,60.58,"alpha:0.03, sampling:smote, batch_size:64, epo..."
1848,IGNnet,82.00,90.60,60.17,60.46,60.31,"alpha:0.01, sampling:smote, batch_size:64, epo..."
1878,IGNnet,82.00,90.60,60.17,60.46,60.31,"alpha:0.01, sampling:smote, batch_size:64, epo..."
1892,IGNnet,82.00,90.60,60.17,60.46,60.31,"alpha:0.01, sampling:smote, batch_size:64, epo..."
1899,IGNnet,82.00,90.60,60.17,60.46,60.31,"alpha:0.01, sampling:smote, batch_size:64, epo..."
1913,IGNnet,82.00,90.60,60.17,60.46,60.31,"alpha:0.01, sampling:smote, batch_size:64, epo..."
2032,IGNnet,82.00,90.60,60.17,60.46,60.31,"alpha:0.01, sampling:smote, batch_size:64, epo..."
2060,IGNnet,82.00,90.60,60.17,60.46,60.31,"alpha:0.01, sampling:smote, batch_size:64, epo..."


##### We have selected models with indices 3089 and 2312 as our best models.
##### Therefore, we will consider theri best hyperparameters as our optimal hyperparameters.
##### Plus, we can see that the SMOTE-based models perform better than the other models.

In [10]:
index1 = Table.index[1] - (test1.shape[0] + test2.shape[0] + test3.shape[0] + test4.shape[0] )
index1

714

In [11]:
index2 = Table.index[10] - (test1.shape[0] + test2.shape[0] + test3.shape[0] + test4.shape[0])
index2

49

In [12]:
length = len(model_list)
length

7

In [13]:
best_table1 = test5.iloc[index1 : index1 + length ].drop(['dataset'], axis = 1)
best_table1

,model,AUC,ACC,Precision,Recall,F1_score,best_params
714,IGNnet,82.54,88.65,58.90,62.83,60.31,"alpha:0.01, sampling:borderline-smote, batch_s..."
715,LogisticRegression,78.75,93.39,56.93,50.77,50.17,"{'C': 1.0, 'class_weight': None, 'dual': False..."
716,DecisionTree,53.38,87.35,52.61,53.38,52.87,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit..."
717,RandomForest,79.81,93.13,56.33,51.11,50.90,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
718,MLP,80.47,91.05,54.56,52.92,53.41,"{'activation': 'relu', 'alpha': 0.0001, 'batch..."
719,SVM,79.89,93.71,46.89,49.97,48.38,"{'C': 1.0, 'break_ties': False, 'cache_size': ..."
720,XGBoost,82.83,92.48,58.19,53.20,54.10,"{'objective': 'binary:logistic', 'base_score':..."


In [14]:
print(best_table1.loc[714, 'best_params'])

alpha:0.01, sampling:borderline-smote, batch_size:64, epochs:100, lr:0.01, test_size:0.2, threshold : 0.4


In [15]:
best_table2 = test5.iloc[index2 : index2 + length ].drop(['dataset'], axis = 1)
best_table2

,model,AUC,ACC,Precision,Recall,F1_score,best_params
49,IGNnet,82.00,90.60,60.17,60.46,60.31,"alpha:0.01, sampling:smote, batch_size:64, epo..."
50,LogisticRegression,78.84,93.51,59.44,50.83,50.25,"{'C': 1.0, 'class_weight': None, 'dual': False..."
51,DecisionTree,52.59,86.77,51.90,52.59,52.08,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit..."
52,RandomForest,80.37,93.06,55.77,51.08,50.85,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
53,MLP,80.51,91.89,57.93,54.35,55.35,"{'activation': 'relu', 'alpha': 0.0001, 'batch..."
54,SVM,79.98,93.71,46.89,49.97,48.38,"{'C': 1.0, 'break_ties': False, 'cache_size': ..."
55,XGBoost,82.14,92.09,54.90,52.02,52.41,"{'objective': 'binary:logistic', 'base_score':..."


In [16]:
print(best_table2.loc[49, 'best_params'])

alpha:0.01, sampling:smote, batch_size:64, epochs:100, lr:0.01, test_size:0.2, threshold : 0.5


##### Now, We compare the performance between methods about oversamplings

#### ALL Performance

In [17]:
sampling_methods = ['adasyn', 'over-random']

In [18]:
IGNNet['sampling_method'] = IGNNet['best_params'].apply(lambda x: next((s for s in sampling_methods if s in str(x)), None))

best_auc_idx = IGNNet.groupby("sampling_method")["AUC"].idxmax()
best_f1_idx = IGNNet.groupby("sampling_method")["F1_score"].idxmax()
best_auc = IGNNet.loc[best_auc_idx]
best_f1 = IGNNet.loc[best_f1_idx]

merged_perform_all = pd.concat([best_auc, best_f1]).drop_duplicates().reset_index(drop=True)

final_rows = []
for method in sampling_methods:
    subset = merged_perform_all[merged_perform_all['sampling_method'] == method]

    if len(subset) > 1:
        row1, row2 = subset.iloc[0], subset.iloc[1]
        if row1["AUC"] > row2["AUC"] and row1["F1_score"] > row2["F1_score"]:
            final_rows.append(row1)
        elif row2["AUC"] > row1["AUC"] and row2["F1_score"] > row1["F1_score"]:
            final_rows.append(row2)
        else:
            subset = subset.copy() 
            subset["mean_score"] = (subset["AUC"] + subset["F1_score"]) / 2
            chosen_row = subset.sort_values("mean_score", ascending=False).iloc[0]
            final_rows.append(chosen_row)
    elif len(subset) == 1:
        final_rows.append(subset.iloc[0])

final_perform_all = pd.DataFrame(final_rows).reset_index(drop=True).drop(['best_params', 'mean_score'], axis = 1)

/tmp/ipykernel_19603/3800498670.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IGNNet['sampling_method'] = IGNNet['best_params'].apply(lambda x: next((s for s in sampling_methods if s in str(x)), None))


In [19]:
best_table1

,model,AUC,ACC,Precision,Recall,F1_score,best_params
714,IGNnet,82.54,88.65,58.90,62.83,60.31,"alpha:0.01, sampling:borderline-smote, batch_s..."
715,LogisticRegression,78.75,93.39,56.93,50.77,50.17,"{'C': 1.0, 'class_weight': None, 'dual': False..."
716,DecisionTree,53.38,87.35,52.61,53.38,52.87,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit..."
717,RandomForest,79.81,93.13,56.33,51.11,50.90,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
718,MLP,80.47,91.05,54.56,52.92,53.41,"{'activation': 'relu', 'alpha': 0.0001, 'batch..."
719,SVM,79.89,93.71,46.89,49.97,48.38,"{'C': 1.0, 'break_ties': False, 'cache_size': ..."
720,XGBoost,82.83,92.48,58.19,53.20,54.10,"{'objective': 'binary:logistic', 'base_score':..."


In [ ]:
best_table1['sampling_method'] = 'borderline-smote'
best_table2['sampling_method'] = 'smote'

In [25]:
pd.concat([best_table1.iloc[[0], :], best_table2.iloc[[0], :], final_perform_all], axis = 0, ignore_index = True).drop(['best_params'], axis = 1)

,model,AUC,ACC,Precision,Recall,F1_score,sampling_method
0,IGNnet,82.54,88.65,58.90,62.83,60.31,borderline-smote
1,IGNnet,82.00,90.60,60.17,60.46,60.31,smote
2,IGNnet,80.31,84.14,57.75,66.33,59.33,adasyn
3,IGNnet,81.60,80.84,57.73,70.28,58.74,over-random
